In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("./upgrade 15 aug 25.xlsx", sheet_name="Sheet1")

In [3]:
groupByProductName = df.groupby("PRODUCT_NAME")

In [4]:
groupByProductName["PRODUCT_NAME"].unique().apply(lambda names: names[0])

PRODUCT_NAME
Banana Leaf                          Banana Leaf
Beans Diced                          Beans Diced
Beetroot                                Beetroot
Bel Patta                              Bel Patta
Bel Patta Toran                  Bel Patta Toran
                                   ...          
Small Green Capsicum        Small Green Capsicum
Tender Coconut                    Tender Coconut
Toran                                      Toran
Tulsi Leaves                        Tulsi Leaves
Yellow Marigold Flower    Yellow Marigold Flower
Name: PRODUCT_NAME, Length: 68, dtype: object

In [ ]:
final = pd.DataFrame(
    {
        # "Product_Name": groupByProductName["PRODUCT_NAME"].unique().apply(lambda names: names[0]),
        "WEIGHT": groupByProductName["WEIGHT"]
        .unique()
        .apply(lambda weights: weights[0]),
        "INDENT": groupByProductName["INDENT"].sum().astype(int),
    }
)
final.reset_index(inplace=True)
final.columns = ["Product_Name", "Weight", "Indents"]

In [ ]:
final["Sr No"] = range(1, len(final) + 1)
final["Final"] = (
    final["Weight"]
    .apply(
        lambda weight: "".join(
            [char for char in weight if 48 <= ord(char) and ord(char) <= 57]
        )
    )
    .astype(int)
    * final["INDENT"]
)
final = pd.concat(
    [
        final,
        pd.DataFrame(
            {
                "Sr No": [""],
                "Product_Name": [""],
                "Weight": [""],
                "Indents": [f'Total: {final["Indents"].sum().astype(int)}'],
                "Final": [""],
            }
        ),
    ]
)

In [ ]:
final[["Sr No", "Product_Name", "Weight", "Indents", "Final"]].to_excel(
    "./../Final.xlsx",
    sheet_name="Final Report",
    index=False,
    engine="openpyxl",
    freeze_panes=(1, 0),
    columns=["Sr No", "Product_Name", "Weight", "Indents", "Final"],
)